In [34]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
# total number of schools
total_schools = school_data['school_name'].count()
# total number of students
total_students = student_data['student_name'].count()
print(total_students)
# total budget
total_budget = school_data['budget'].sum()
# average math score
average_math_score = round(school_data_complete['math_score'].mean(), 6)
# average reading score
average_reading_score = round(school_data_complete['reading_score'].mean(), 6)
# percentage of students with passing math score (70+)
plus70_math = 0
for score in school_data_complete['math_score']:
    if score >= 70:
        plus70_math = plus70_math + 1

percent_math_passing = round(((plus70_math / total_students) * 100), 6)
# percentage of students with passing reading score (70+)
plus70_reading = 0
for score in school_data_complete['reading_score']:
    if score >= 70:
        plus70_reading = plus70_reading + 1

percent_reading_passing = round(((plus70_reading / total_students) * 100), 6)

39170


In [37]:
plus70_reading_math = 0
reading_math_combined = pd.DataFrame(school_data_complete.loc[:,["reading_score", "math_score"]])
for x,y in reading_math_combined.itertuples():
    if x >= 70 and y >= 70:
        plus70_reading_math = plus70_reading_math + 1
        
print(plus70_reading_math)
# for row in reading_math_combined:
#     print(row)
#     if(row[1].value() >= 70 and row[2].value() >= 70):
#         plus70_combined = plus70_combined +1

# print(plus70_combined)

ValueError: too many values to unpack (expected 2)

In [5]:
# percentage of students who passed math *and* reading (% Overall Passing)
plus70_reading_math = 0
for score in school_data_complete:
    score['reading_score']
print(plus70_reading_math)
# dataframe to hold above results

987125700


In [44]:
# dataframe for district summary
district_summary = pd.DataFrame({"Total Schools" : total_schools,"Total Students" : total_students,"Total Budget" : total_budget,"Average Math Score" : average_math_score,"Average Reading Score" : average_reading_score,"% Passing Math" : percent_math_passing,"% Passing Reading" : percent_reading_passing,"% Passing Overall" : "tbd"}, index=[0])
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,tbd


In [40]:
School Summary
Create an overview table that summarizes key metrics about each school, including:

School Name
School Type
Total Students
Total School Budget
Per Student Budget
Average Math Score
Average Reading Score
% Passing Math
% Passing Reading
% Overall Passing (The percentage of students that passed math and reading.)
Create a dataframe to hold the above results

SyntaxError: invalid syntax (3449900226.py, line 1)